In [2]:
!huggingface-cli login

^C


In [3]:
from huggingface_hub import HfApi

# Load the token from the file
with open("C:/Users/adity/.cache/huggingface/token", "r") as f:
    token = f.read().strip()

# Use the token for authentication
api = HfApi(token=token)

c:\Users\adity\CSE8803DLT-24Fall-Assignment2-Public\cuda_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "facebook/layerskip-llama3.2-1B"  # Example
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

c:\Users\adity\CSE8803DLT-24Fall-Assignment2-Public\cuda_env\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--facebook--layerskip-llama3.2-1B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [7]:
save_directory = "../llm-transparency-tool/llm_transparency_tool/models/layerskip-llama3.2-1B"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('../llm-transparency-tool/llm_transparency_tool/models/layerskip-llama3.2-1B\\tokenizer_config.json',
 '../llm-transparency-tool/llm_transparency_tool/models/layerskip-llama3.2-1B\\special_tokens_map.json',
 '../llm-transparency-tool/llm_transparency_tool/models/layerskip-llama3.2-1B\\tokenizer.json')

In [6]:
print(tokenizer.name_or_path)
print(model.name_or_path)  # This will show the model's cache path

facebook/layerskip-llama3.2-1B
facebook/layerskip-llama3.2-1B


In [7]:
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

model = AutoModelForCausalLM.from_pretrained(model_name, local_files_only = True).to(device)

In [9]:
print(device)

cuda


In [19]:
# Example input text
input_text = "What is the capital of France?"

# Tokenize the input and move to GPU
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model.generate(**inputs, max_new_tokens=400)

# Decode the output tokens to text
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(output_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is the capital of France? A. Paris B. New Orleans C. Washington D. Montpelier
A. 8
B. 9
C. 10
D. 11
Answer: A
Explanation:   Paris is the capital of France.
The capital of France is Paris.


In [16]:
import pandas as pd


In [17]:
df = pd.read_csv("train.csv")

In [20]:
!pip install datasets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("cnn_dailymail", "3.0.0") 

c:\Users\adity\CSE8803DLT-24Fall-Assignment2-Public\cuda_env\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\datasets--cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 147212.62 examples/s]


In [26]:
system_prompt = "Summarize the given article in a concise manner. Output Format Should be: Input : <input_text> \n Output : <output_text>"

In [28]:
# Function to generate summaries
def summarize(text, model, tokenizer, system_prompt):
    # Combine the system prompt and the input text
    input_text = f"{system_prompt}\n{text}"
    
    # Tokenize the input and move to the correct device
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    # Generate the summary
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=400,  # Adjust based on your needs
            num_beams=5,         # Beam search for better results
            early_stopping=True,
        )
    
    # Decode the output and extract the summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Summarize the first 100 rows
summaries = []
for i, example in enumerate(dataset["train"]):
    if i >= 5:  # Stop after 100 rows
        break
    article = example["article"]
    summary = summarize(article, model, tokenizer, system_prompt)
    summaries.append((article, summary))
    print(f"Processed row {i + 1}: {summary}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed row 1: Summarize the given article in a concise manner. Output Format Should be: Input : <input_text> 
 Output : <output_text>
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed row 2: Summarize the given article in a concise manner. Output Format Should be: Input : <input_text> 
 Output : <output_text>
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed row 3: Summarize the given article in a concise manner. Output Format Should be: Input : <input_text> 
 Output : <output_text>
MINNEAPOLIS, Minnesota (CNN) -- Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. "The whole bridge from one side of the Mississippi to the other just completely gave way, fell all the way down," survivor Gary Babineau told CNN. "I probably had a 30-, 35-foot free fall. And there's cars in the water, there's cars on fire. The whole bridge is down." He said his back was injured but he determined he could move around. "I realized there was a school bus right next to me, and me and a couple of other guys went over and started lifting the kids off the bridge. They were yelling, screaming, bleeding. I think there were some broken bones."  Watch a driver describe his narrow escape ». At home when he heard about the disaster, Dr. John Hink, an emergency room physician, jumped into his car and rushed to the scene i

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Processed row 4: Summarize the given article in a concise manner. Output Format Should be: Input : <input_text> 
 Output : <output_text>
WASHINGTON (CNN) -- Doctors removed five small polyps from President Bush's colon on Saturday, and "none appeared worrisome," a White House spokesman said. The polyps were removed and sent to the National Naval Medical Center in Bethesda, Maryland, for routine microscopic examination, spokesman Scott Stanzel said. Results are expected in two to three days. All were small, less than a centimeter [half an inch] in diameter, he said. Bush is in good humor, Stanzel said, and will resume his activities at Camp David. During the procedure Vice President Dick Cheney assumed presidential power. Bush reclaimed presidential power at 9:21 a.m. after about two hours. Doctors used "monitored anesthesia care," Stanzel said, so the president was asleep, but not as deeply unconscious as with a true general anesthetic. He spoke to first lady Laura Bush -- who is in Mi

In [29]:
with open("summaries2.txt", "w") as f:
    for i, (article, summary) in enumerate(summaries):
        f.write(f"Row {i + 1}:\n")
        f.write(f"Article: {article}\n")
        f.write(f"Summary: {summary}\n")
        f.write("\n")

In [ ]:
with open("summaries.txt", "w") as f:
    for i, (article, summary) in enumerate(summaries):
        f.write(f"Row {i + 1}:\n")
        f.write(f"Article: {article}\n")
        f.write(f"Summary: {summary}\n")
        f.write("\n")